In [ ]:
load_model = "0819"
save_model = "0221-2"

In [2]:
from fennet.mhc.mhc_binding_model import *
from fennet.mhc.mhc_utils import *

fasta_list = ["uniprotkb_UP000005640_AND_reviewed_true_2024_03_01.fasta"]

test_allele_list = [
    "B41_02",
]

import pandas as pd

hla_df, hla_esm_list = load_esm_pkl(fname="embeds/hla1_esm_embeds.pkl")
train_df = pd.read_excel("all_alleles/train_data/systemhc_fine_tune/train_df_20%.xlsx")
test_df = pd.read_excel("all_alleles/train_data/systemhc_fine_tune/test_df.xlsx")

In [3]:
pept_encoder = ModelSeqEncoder()
hla_encoder = ModelHlaEncoder()

hla_encoder.load_state_dict(
    torch.load(f"model/HLA_model_v{load_model}.pt", map_location="cuda")
)
pept_encoder.load_state_dict(
    torch.load(f"model/pept_model_v{load_model}.pt", map_location="cuda")
)

dataset = HlaDataSet(hla_df, hla_esm_list, train_df, fasta_list)

In [5]:
import neptune.new as neptune

run = neptune.init_run(
    project="FennOmix/FeNNetMHC",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJmOThhZjVjNS1jMzQ3LTQ5MGUtOWI3YS0zYTU0ZjE0NTI0ZDYifQ==",
    name="test fine tuning with 20% data",
    tags=["HLA1"],
)

test_bundle = (test_df, test_allele_list, hla_df, hla_esm_list, fasta_list)

train(
    hla_encoder,
    pept_encoder,
    dataset,
    epoch=20,
    warmup_epoch=5,
    batch_size=20,
    lr=1e-4,
    verbose=True,
    device="cuda",
    test_bundle=test_bundle,
    neptune_run=run,
)

[neptune] [info   ] Neptune initialized. Open in the app: https://app.neptune.ai/FennOmix/FeNNetMHC/e/HLAL-25
2025-02-24 15:42:00> 118 training samples
2025-02-24 15:42:01> [Epoch=0] loss=0.04845, lr=2.000e-05
test alleles rank%<0.1 average recall rate: 0.34
test alleles rank%<0.5 average recall rate: 0.92
test alleles rank%<2 average recall rate: 0.99
2025-02-24 15:42:36> [Epoch=1] loss=0.04723, lr=4.000e-05
test alleles rank%<0.1 average recall rate: 0.48
test alleles rank%<0.5 average recall rate: 0.92
test alleles rank%<2 average recall rate: 0.99
2025-02-24 15:43:13> [Epoch=2] loss=0.04430, lr=6.000e-05
test alleles rank%<0.1 average recall rate: 0.34
test alleles rank%<0.5 average recall rate: 0.92
test alleles rank%<2 average recall rate: 0.99
2025-02-24 15:43:49> [Epoch=3] loss=0.04329, lr=8.000e-05
test alleles rank%<0.1 average recall rate: 0.64
test alleles rank%<0.5 average recall rate: 0.96
test alleles rank%<2 average recall rate: 0.97
2025-02-24 15:44:25> [Epoch=4] loss=

In [6]:
torch.save(hla_encoder.state_dict(), f"model/HLA_model_v{save_model}.pt")
torch.save(pept_encoder.state_dict(), f"model/pept_model_v{save_model}.pt")